<a href="https://colab.research.google.com/github/Melody-Wang/linguistic_abstraction/blob/main/projection_study_keybert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

!pip install keybert
from keybert import KeyBERT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.3 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=c29e713d798500ff01e2123b220308acbca78569ffc54ef41cb088d2d5065a40
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=7758e8ee6b81ebea9131cbb465b3d3002733f0150212b8

In [2]:
data_dir = '/content/drive/MyDrive/Linguistic abstraction summer 2023/Datasets/Raw text data/'

In [3]:
proj = pd.read_csv(data_dir+'projection study for KeyBERT 10.25.23.csv', index_col=0)

In [4]:
proj.head(3)

,1_s_w,1_s_h,2_s_w,2_s_h,3_s_w,3_s_h,4_s_w,4_s_h,5_s_w,5_s_h,...,6_o_h,7_o_w,7_o_h,8_o_w,8_o_h,9_o_w,9_o_h,10_o_w,10_o_h,manip_check
1,To lose weight,"Running, Pilates","Makes me happy, gives me background noise",On Spotify,Entertainment,"HBO Max, Apple TV, Netflix",To spend time with friends,I go to various restaurants,"To keep up with friends, to laugh","TikTok, Instagram, Twitter",...,Online,New experinces,Road trips,Happiness,Hanging out,Happiness,Hanging out,To learn,Spotifty,4
2,I enjoy learning.,Mainly on Audible.,I enjoy the feeling it gives me.,I listen on my phone while running errands.,I enjoy playing and the challenge.,By deciding what friends/ family want to play.,Look up what I'm interested in.,Opening my phone or computer.,I enjoy good food.,By seeing what looks fresh in the market and c...,...,Drive to the grocery store.,To get what is needed.,Online,To see new cities.,By car.,Close in proximity.,"Holidays, lives with them, TV",Likes the friendship,"By having dinner, playing games, movies, etc",4
3,it is fun,spotify on my phone,to be entertained,netflix on my laptop,to eat yummy food,go to restaurants,when i am bored,using my phone or laptop,to see fun videos and pictures,through my phone,...,go to soho and look around,to experience new places,plane or car,to feel good,eat together,because it is fun,eat and study together,to be entertained,through their laptop,2


In [5]:
kw_model = KeyBERT(model='all-mpnet-base-v2')

#### Preprocessing

In [6]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [7]:
def preprocess(x):
  if type(x) is not str: x = str(x)
  x = x.lower()
  x = re.sub('[^0-9A-Za-z ]', '' , x)
  return [lemmatizer.lemmatize(c) for c in word_tokenize(x) if c not in stop_words]

In [8]:
proj_processed = proj.apply(lambda c: c.apply(preprocess)).apply(lambda c: c.apply(lambda x: ' '.join(x)))
proj_processed.head(3)

,1_s_w,1_s_h,2_s_w,2_s_h,3_s_w,3_s_h,4_s_w,4_s_h,5_s_w,5_s_h,...,6_o_h,7_o_w,7_o_h,8_o_w,8_o_h,9_o_w,9_o_h,10_o_w,10_o_h,manip_check
1,lose weight,running pilate,make happy give background noise,spotify,entertainment,hbo max apple tv netflix,spend time friend,go various restaurant,keep friend laugh,tiktok instagram twitter,...,online,new experinces,road trip,happiness,hanging,happiness,hanging,learn,spotifty,4
2,enjoy learning,mainly audible,enjoy feeling give,listen phone running errand,enjoy playing challenge,deciding friend family want play,look im interested,opening phone computer,enjoy good food,seeing look fresh market coming recipe idea,...,drive grocery store,get needed,online,see new city,car,close proximity,holiday life tv,like friendship,dinner playing game movie etc,4
3,fun,spotify phone,entertained,netflix laptop,eat yummy food,go restaurant,bored,using phone laptop,see fun video picture,phone,...,go soho look around,experience new place,plane car,feel good,eat together,fun,eat study together,entertained,laptop,2


#### Keyword Extraction with KeyBert

In [9]:
#average sentence length across dataset
proj_processed.apply(lambda c: c.apply(lambda x: len(str(x).split(' ')))).apply(lambda c: c.mean()).mean()

3.7456808943089435

In [10]:
def extract_kw(x, ngram_range=(1,1), top_n=3):
  return kw_model.extract_keywords(x, keyphrase_ngram_range=ngram_range, stop_words='english', top_n=top_n)

In [11]:
proj_kw = proj_processed.apply(lambda c: c.apply(extract_kw)).apply(lambda c: c.apply(lambda x: ' '.join(list(dict(x).keys()))))

In [12]:
proj_kw = proj_kw.drop(columns = ['manip_check'], axis=0)
proj_kw.columns = [c+'_keybert_3' for c in proj_kw.columns]

In [13]:
proj_kw.head(3)

,1_s_w_keybert_3,1_s_h_keybert_3,2_s_w_keybert_3,2_s_h_keybert_3,3_s_w_keybert_3,3_s_h_keybert_3,4_s_w_keybert_3,4_s_h_keybert_3,5_s_w_keybert_3,5_s_h_keybert_3,...,6_o_w_keybert_3,6_o_h_keybert_3,7_o_w_keybert_3,7_o_h_keybert_3,8_o_w_keybert_3,8_o_h_keybert_3,9_o_w_keybert_3,9_o_h_keybert_3,10_o_w_keybert_3,10_o_h_keybert_3
1,weight lose,pilate running,noise happy background,spotify,entertainment,hbo netflix tv,friend spend time,restaurant various,friend laugh,tiktok instagram twitter,...,clothes new,online,experinces new,trip road,happiness,hanging,happiness,hanging,learn,spotifty
2,learning enjoy,audible mainly,feeling enjoy,phone listen errand,challenge playing enjoy,play family friend,interested im look,phone computer opening,food enjoy good,recipe fresh market,...,eat able home,grocery drive store,needed,online,city new,car,proximity close,tv holiday life,friendship like,dinner playing game
3,fun,spotify phone,entertained,netflix laptop,eat food yummy,restaurant,bored,phone laptop using,video fun picture,phone,...,new thing,soho look,place experience new,plane car,feel good,eat,fun,study eat,entertained,laptop


#### Compute Similarity 

In [14]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

In [16]:
proj_processed = proj_processed.drop(columns=['manip_check'])

In [21]:
#use sentence transformer to create embeddings
text_embeddings = proj_processed.apply(lambda c: c.apply(lambda x: model.encode(x))).to_numpy()
kw_embeddings = proj_kw.apply(lambda c: c.apply(lambda x: model.encode(x))).to_numpy()

In [18]:
text_embeddings = text_embeddings.reshape(proj_processed.shape[0], proj_processed.shape[1])
kw_embeddings = kw_embeddings.reshape(proj_processed.shape[0], proj_processed.shape[1])

In [25]:
text_embeddings = np.hstack(text_embeddings.reshape(-1)).reshape(proj_processed.shape[0], proj_processed.shape[1], -1)
kw_embeddings = np.hstack(kw_embeddings.reshape(-1)).reshape(proj_processed.shape[0], proj_processed.shape[1], -1)

In [26]:
text_embeddings.shape

(96, 40, 384)

In [34]:
scores = np.array([cosine_similarity(text_embeddings[i,j,:].reshape(1,-1), kw_embeddings[i,j,:].reshape(1,-1)) for i in range(proj_processed.shape[0]) for j in range(proj_processed.shape[1])])

In [39]:
scores = scores.reshape(proj_processed.shape[0], proj_processed.shape[1], -1).squeeze()

In [49]:
sim1_scores = pd.DataFrame(scores, columns=[c+'_sim1' for c in proj_kw.columns])

In [50]:
sim1_scores.head(3)

,1_s_w_keybert_3_sim1,1_s_h_keybert_3_sim1,2_s_w_keybert_3_sim1,2_s_h_keybert_3_sim1,3_s_w_keybert_3_sim1,3_s_h_keybert_3_sim1,4_s_w_keybert_3_sim1,4_s_h_keybert_3_sim1,5_s_w_keybert_3_sim1,5_s_h_keybert_3_sim1,...,6_o_w_keybert_3_sim1,6_o_h_keybert_3_sim1,7_o_w_keybert_3_sim1,7_o_h_keybert_3_sim1,8_o_w_keybert_3_sim1,8_o_h_keybert_3_sim1,9_o_w_keybert_3_sim1,9_o_h_keybert_3_sim1,10_o_w_keybert_3_sim1,10_o_h_keybert_3_sim1
0,0.963862,0.978207,0.784662,1.000000,1.000000,0.857796,0.908184,0.885166,0.840475,1.000000,...,0.969936,1.000000,0.969785,0.933527,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
1,0.945073,0.985170,0.789955,0.949923,0.928920,0.840117,0.944554,0.973741,0.936939,0.831056,...,0.959851,0.957931,0.707201,1.000000,0.906177,1.000000,0.965319,0.925217,0.975403,0.87131
2,1.000000,1.000000,1.000000,1.000000,0.977015,0.818676,1.000000,0.954238,0.919037,1.000000,...,0.775692,0.912425,0.958424,1.000000,1.000000,0.742679,1.000000,0.877829,1.000000,1.00000


In [51]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [52]:
text_embeddings = proj_processed.apply(lambda c: c.apply(lambda x: model.encode(x))).to_numpy()
kw_embeddings = proj_kw.apply(lambda c: c.apply(lambda x: model.encode(x))).to_numpy()

In [53]:
text_embeddings = text_embeddings.reshape(proj_processed.shape[0], proj_processed.shape[1])
kw_embeddings = kw_embeddings.reshape(proj_processed.shape[0], proj_processed.shape[1])

In [54]:
text_embeddings = np.hstack(text_embeddings.reshape(-1)).reshape(proj_processed.shape[0], proj_processed.shape[1], -1)
kw_embeddings = np.hstack(kw_embeddings.reshape(-1)).reshape(proj_processed.shape[0], proj_processed.shape[1], -1)

In [55]:
scores = np.array([cosine_similarity(text_embeddings[i,j,:].reshape(1,-1), kw_embeddings[i,j,:].reshape(1,-1)) for i in range(proj_processed.shape[0]) for j in range(proj_processed.shape[1])])
scores = scores.reshape(proj_processed.shape[0], proj_processed.shape[1], -1).squeeze()

In [58]:
sim2_scores = pd.DataFrame(scores, columns=[c+'_sim2' for c in proj_kw.columns])

In [59]:
sim2_scores.head(3)

,1_s_w_keybert_3_sim2,1_s_h_keybert_3_sim2,2_s_w_keybert_3_sim2,2_s_h_keybert_3_sim2,3_s_w_keybert_3_sim2,3_s_h_keybert_3_sim2,4_s_w_keybert_3_sim2,4_s_h_keybert_3_sim2,5_s_w_keybert_3_sim2,5_s_h_keybert_3_sim2,...,6_o_w_keybert_3_sim2,6_o_h_keybert_3_sim2,7_o_w_keybert_3_sim2,7_o_h_keybert_3_sim2,8_o_w_keybert_3_sim2,8_o_h_keybert_3_sim2,9_o_w_keybert_3_sim2,9_o_h_keybert_3_sim2,10_o_w_keybert_3_sim2,10_o_h_keybert_3_sim2
0,0.964095,0.954786,0.888054,1.000000,1.000000,0.801095,0.814599,0.852329,0.831238,1.00000,...,0.971418,1.000000,0.952779,0.894178,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000
1,0.883333,0.972933,0.771276,0.905364,0.896060,0.810385,0.871512,0.946474,0.939125,0.84378,...,0.930949,0.976011,0.627150,1.000000,0.86436,1.00000,0.977943,0.917439,0.947789,0.774275
2,1.000000,1.000000,1.000000,1.000000,0.987004,0.808379,1.000000,0.969741,0.867237,1.00000,...,0.735666,0.796195,0.927016,1.000000,1.00000,0.72289,1.000000,0.834876,1.000000,1.000000


In [78]:
proj_processed.columns = [c+'_Processed' for c in proj.columns[:-1]]
sim1_scores.index = proj.index
sim2_scores.index = proj.index

In [81]:
proj_keybert_results = pd.concat([proj, proj_processed, proj_kw, sim1_scores, sim2_scores], axis=1)

In [82]:
proj_keybert_results.to_csv('/content/drive/MyDrive/projection_study_keybert_results.csv')